# Reciprocal-Rank-Fusion outperforms Condorcet and individual Rank Learning Methods

https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf


In [2]:
%pip install llama-index-llms-openai
%pip install llama-index-retrievers-bm25

Note: you may need to restart the kernel to use updated packages.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pystemmer: filename=PyStemmer-2.2.0.1-cp310-cp310-macosx_12_0_arm64.whl size=221731 sha256=fe86dbcec8723cd960057a4a3862cf44f80e6059acb22be382408f552d14d4ae
  Stored in directory: /Users/ngocp/Library/Caches/pip/wheels/45/7d/2c/a7ebb8319e01acc5306fa1f8558bf24063d6cec2c02de330c9
Successfully built pystemmer
Note: you may need to restart the kernel to use updated packages.


In [1]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("../data/").load_data()

In [3]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=256)

index = VectorStoreIndex.from_documents(documents, transformations=[splitter])

In [4]:
from llama_index.retrievers.bm25 import BM25Retriever

vector_retriever = index.as_retriever(similarity_top_k=2)

bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=2
)

/Users/ngocp/Documents/projects/tu-vi-bot/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    similarity_top_k=2,
    num_queries=4,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    # query_gen_prompt="...",  # we could override the query generation prompt here
)

In [6]:
# apply nested async to run in a notebook
import nest_asyncio

nest_asyncio.apply()

In [7]:
nodes_with_scores = retriever.retrieve(
    "Các cunh mệnh nào phù hợp với cơ nghiệp ?"
)

print(nodes_with_scores)

Generated queries:
1. Các cung mệnh phù hợp với ngành nghề nào?
2. Cách chọn cung mệnh phù hợp với công việc?
3. Cung mệnh nào thích hợp với sở thích và năng lực của mình?
[NodeWithScore(node=TextNode(id_='1ebd8ef3-2832-40f9-9896-a41f42e71390', embedding=None, metadata={'file_path': '/Users/ngocp/Documents/projects/tu-vi-bot/research/../data/cung-menh.txt', 'file_name': 'cung-menh.txt', 'file_type': 'text/plain', 'file_size': 406165, 'creation_date': '2024-10-03', 'last_modified_date': '2024-10-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e169f7ae-7108-4a0f-86fb-bc911ad8c0ae', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/Users/ngocp/Documents/projects/tu-vi-bot/research/../data/cu

In [8]:
for node in nodes_with_scores:
    print(f"Score: {node.score:.2f} - {node.text}...\n-----\n")


Score: 0.05 - - Cự Môn độc tọa cung Tị hoặc Hợi, vì cung quan lộc là Thiên Cơ nên khá thích hợp với các công việc về văn và lập kế hoạch.
- Cự Môn ở cung Thìn hoặc Tuất, cung quan lộc là Thái Âm; trường hợp Thái Dương ở cung Thìn là cung vượng, thích hợp làm công chức hay giảng sư; trường hợp Thái Dương ở cung Tuất thì làm công chức là bất lợi, có thể làm nghề môi giới bảo hiểm, tổ chức sự kiện, chủ trì hội trường....
-----

Score: 0.03 - thích hợp với những nghề nghiệp cần phải động khẩu, động não, có tính cạnh tranh cao độ như thầy giáo, giáo sư, luật sư, quảng bá, mở rộng thị trường, nhân viên bán hàng, văn hóa, xuất bản, hoặc mậu dịch xuất nhập khẩu, đại lí bán hàng, môi giới mua bán chuyên nghiệp, quan hệ công cộng, bảo hiểm, hướng dẫn viên du lịch nghệ thuật biểu diễn, ẩm thực, v.v. đều rất thích hợp. Nhưng Cự Môn dễ bị bài xích, cần phải tránh họa xuất phát từ cái miệng....
-----



In [9]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever)

In [12]:
response = query_engine.query(
    "Các cunh mệnh nào phù hợp với cơ nghiệp ?"
)

print(response)

Generated queries:
1. Các cung mệnh nào phù hợp với ngành nghề kinh doanh?
2. Tìm hiểu về cung mệnh phù hợp với cơ nghiệp
3. Cách xác định cung mệnh phù hợp với lĩnh vực công việc
Các cung mệnh phù hợp với ngành nghề kinh doanh giải trí, công nghiệp nghỉ mát, du lịch, giải trí, quan hệ công cộng, nghệ thuật biểu diễn bao gồm Thiên Đồng đồng cung với Kình Dương. Các cung mệnh thích hợp với công việc về văn và lập kế hoạch bao gồm Cự Môn độc tọa cung Tị hoặc Hợi, cung Thìn hoặc Tuất với cung quan lộc là Thái Âm, và trường hợp Thái Dương ở cung Thìn hoặc Tuất.
